# NLP - Equity Research Analysis

In [1]:
!pip install torch transformers sentence-transformers langchain langchain_community langchain-huggingface langchain_experimental langchain_chroma langchainhub streamlit unstructured faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

## Scraping latest articles in web related to finance and stocks

In [2]:
import re
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [3]:
query = "latest Financial News Outlets, Stock Market News, Financial Data Providers, Company Earnings Reports, Brokerage Research"
url = f"https://duckduckgo.com/html/?q={query}"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
urls = []

# Extract result links
for result in soup.find_all('a', class_='result__url'):
    # print(result.get('href'))
    urls.append("https:" + result.get('href'))

urls

['https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.marketwatch.com%2F&rut=ccfb3a40f50b5c52867caf6a33b53da5d7fd38a4e85c9440763b7c572bdd1dbc',
 'https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.bloomberg.com%2F&rut=69b84c78f742a3cf7fc533f450d4144ddd3219258dbf49c78622d389e41f00c5',
 'https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.wsj.com%2F&rut=d52bda4c4d03738057357a20e335c4f5f7c36f49e9d4a3d781ff31a4a75667e2',
 'https://duckduckgo.com/l/?uddg=https%3A%2F%2Ffinance.yahoo.com%2F&rut=c7a42c7bdbb3ef83974b2f2a5e6aabe83da19481b721adc17efaf81fef1daf2b',
 'https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.google.com%2Ffinance%2F&rut=252ac5565b096704cb33344d60f45939b769408a37b208a89719e876c228ab5f',
 'https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.wsj.com%2Flivecoverage%2Fcpi%2Djpmorgan%2Dciti%2Dgoldman%2Dearnings%2Dstock%2Dmarket%2D01%2D15%2D2025&rut=b88c778b0f3738378676e558f09fed390d80aeab4353b93f4d3e0c9021a1a759',
 'https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.marketwatch.com%2Fmarkets%2

In [4]:
# Load embeddings model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert URLs or titles to embeddings
embeddings = model.encode(urls)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance
index.add(np.array(embeddings))

# Save URLs with embeddings
url_to_index = {i: url for i, url in enumerate(urls)}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def search_relevant_urls(query, index, model, url_to_index, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = [url_to_index[idx] for idx in indices[0]]
    return results

# Example Query
query = "latest Financial News Outlets, Stock Market News, Financial Data Providers, Company Earnings Reports, Brokerage Research"
relevant_urls = search_relevant_urls(query, index, model, url_to_index)
# print(relevant_urls)
for url in relevant_urls:
  print(url)

https://duckduckgo.com/l/?uddg=https%3A%2F%2Fapnews.com%2Farticle%2Fstock%2Dmarkets%2Dinflation%2Ddata%2Dearnings%2D42efcede3bc3db358117c72f2504edf2&rut=8e729d50597ecce60202f87db63eb8623307c82034d712145630018b2271a0e2
https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.wsj.com%2Flivecoverage%2Fcpi%2Djpmorgan%2Dciti%2Dgoldman%2Dearnings%2Dstock%2Dmarket%2D01%2D15%2D2025&rut=b88c778b0f3738378676e558f09fed390d80aeab4353b93f4d3e0c9021a1a759
https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.marketwatch.com%2Fmarkets%2Ffinancial%2Dmarkets&rut=9e0c593042da93a1df19f84a2c02d4a448d72ffe3e1f1da12193a88a5cebfbce
https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.marketwatch.com%2F&rut=ccfb3a40f50b5c52867caf6a33b53da5d7fd38a4e85c9440763b7c572bdd1dbc
https://duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.bloomberg.com%2F&rut=69b84c78f742a3cf7fc533f450d4144ddd3219258dbf49c78622d389e41f00c5


In [6]:
!pip install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 10.4 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=9676ea7e96d3495a0be76309dc3cd503d11b4f7c059585f1276df9f556dee2c3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=d0b4d37e4803576bbf5c691981aa027dfaa9c689e8b918677de101fac1f75156
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1e

In [7]:
!pip install lxml[html_clean]

In [8]:
import urllib.parse

def extract_final_url(duckduckgo_url):
    parsed_url = urllib.parse.urlparse(duckduckgo_url)
    query_params = urllib.parse.parse_qs(parsed_url.query)
    final_url = query_params.get('uddg', [None])[0]
    return final_url

resolved_urls = [extract_final_url(url) for url in relevant_urls]

# Print the resolved URLs to check
for url in resolved_urls:
    print(url)


https://apnews.com/article/stock-markets-inflation-data-earnings-42efcede3bc3db358117c72f2504edf2
https://www.wsj.com/livecoverage/cpi-jpmorgan-citi-goldman-earnings-stock-market-01-15-2025
https://www.marketwatch.com/markets/financial-markets
https://www.marketwatch.com/
https://www.bloomberg.com/


## Extracting data from relavant URLs

In [9]:
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.document_loaders import NewsURLLoader

loader = NewsURLLoader(urls=resolved_urls)
data = loader.load()
print(len(data))
data

ERROR:langchain_community.document_loaders.news:Error fetching or processing https://www.wsj.com/livecoverage/cpi-jpmorgan-citi-goldman-earnings-stock-market-01-15-2025, exception: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/livecoverage/cpi-jpmorgan-citi-goldman-earnings-stock-market-01-15-2025 on URL https://www.wsj.com/livecoverage/cpi-jpmorgan-citi-goldman-earnings-stock-market-01-15-2025
ERROR:langchain_community.document_loaders.news:Error fetching or processing https://www.marketwatch.com/markets/financial-markets, exception: Article `download()` failed with 401 Client Error: HTTP Forbidden for url: https://www.marketwatch.com/markets/financial-markets on URL https://www.marketwatch.com/markets/financial-markets
ERROR:langchain_community.document_loaders.news:Error fetching or processing https://www.marketwatch.com/, exception: Article `download()` failed with 401 Client Error: HTTP Forbidden for url: https://www.marketwatch.com/ on 

2


[Document(metadata={'title': 'Stock market today: Wall Street drifts lower as momentum slows for US stocks', 'link': 'https://apnews.com/article/stock-markets-inflation-data-earnings-42efcede3bc3db358117c72f2504edf2', 'authors': [], 'language': 'en', 'description': 'U.S. stock indexes drifted lower following a mixed set of earnings reports from Morgan Stanley, UnitedHealth Group and other big companies.', 'publish_date': datetime.datetime(2025, 1, 16, 5, 20, 34)}, page_content='NEW YORK (AP) — U.S. stock indexes drifted lower Thursday following a mixed set of earnings reports from Morgan Stanley, UnitedHealth Group and other big companies.\n\nThe S&P 500 slipped 0.2% after flipping between small gains and losses through the day. More stocks rose within the index than fell, but drops for some influential stocks like Tesla outweighed them.\n\nThe Dow Jones Industrial Average dropped 68 points, or 0.2%, and the Nasdaq composite fell 0.9%.\n\nThe relatively modest moves for stocks came a d

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
docs = text_splitter.split_documents(data)

print("Total number of documents: ",len(docs))

Total number of documents:  13


In [11]:
docs

[Document(metadata={'title': 'Stock market today: Wall Street drifts lower as momentum slows for US stocks', 'link': 'https://apnews.com/article/stock-markets-inflation-data-earnings-42efcede3bc3db358117c72f2504edf2', 'authors': [], 'language': 'en', 'description': 'U.S. stock indexes drifted lower following a mixed set of earnings reports from Morgan Stanley, UnitedHealth Group and other big companies.', 'publish_date': datetime.datetime(2025, 1, 16, 5, 20, 34)}, page_content='NEW YORK (AP) — U.S. stock indexes drifted lower Thursday following a mixed set of earnings reports from Morgan Stanley, UnitedHealth Group and other big companies.\n\nThe S&P 500 slipped 0.2% after flipping between small gains and losses through the day. More stocks rose within the index than fell, but drops for some influential stocks like Tesla outweighed them.\n\nThe Dow Jones Industrial Average dropped 68 points, or 0.2%, and the Nasdaq composite fell 0.9%.'),
 Document(metadata={'title': 'Stock market toda

## Making the data retrieval-ready!

In [12]:
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.8 MB/s eta 0:00:00


In [13]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GenAI_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("test embeddings")
# vector

In [14]:
from langchain_community.vectorstores.utils import filter_complex_metadata

# Filter complex metadata from the documents
docs = filter_complex_metadata(docs)

print(docs)
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

[Document(metadata={'title': 'Stock market today: Wall Street drifts lower as momentum slows for US stocks', 'link': 'https://apnews.com/article/stock-markets-inflation-data-earnings-42efcede3bc3db358117c72f2504edf2', 'language': 'en', 'description': 'U.S. stock indexes drifted lower following a mixed set of earnings reports from Morgan Stanley, UnitedHealth Group and other big companies.'}, page_content='NEW YORK (AP) — U.S. stock indexes drifted lower Thursday following a mixed set of earnings reports from Morgan Stanley, UnitedHealth Group and other big companies.\n\nThe S&P 500 slipped 0.2% after flipping between small gains and losses through the day. More stocks rose within the index than fell, but drops for some influential stocks like Tesla outweighed them.\n\nThe Dow Jones Industrial Average dropped 68 points, or 0.2%, and the Nasdaq composite fell 0.9%.'), Document(metadata={'title': 'Stock market today: Wall Street drifts lower as momentum slows for US stocks', 'link': 'http

In [15]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("what is the price of tiago iCNG?")
print('Length of retrieved documents: ', len(retrieved_docs))
print(retrieved_docs)
print(retrieved_docs[1].page_content)

Length of retrieved documents:  3
[Document(id='c1428005-da45-40a2-84f5-74579caec8fd', metadata={'description': '', 'language': 'en', 'link': 'https://www.bloomberg.com/', 'title': 'Are you a robot?'}, page_content='Why did this happen?\n\nPlease make sure your browser supports JavaScript and cookies and that you are not blocking them from loading. For more information you can review our Terms of Service and Cookie Policy.'), Document(id='b4c2bd03-5a0b-44ce-8f27-454518609256', metadata={'description': 'U.S. stock indexes drifted lower following a mixed set of earnings reports from Morgan Stanley, UnitedHealth Group and other big companies.', 'language': 'en', 'link': 'https://apnews.com/article/stock-markets-inflation-data-earnings-42efcede3bc3db358117c72f2504edf2', 'title': 'Stock market today: Wall Street drifts lower as momentum slows for US stocks'}, page_content='It was the company’s first financial report since the shooting of one of its executives outside a New York City hotel e

## Performing RAG

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an expert equity research analyst. Use the following pieces of context to answer the question."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [18]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

## Output Evaluation

In [19]:
response = rag_chain.invoke({"input": "how is Tech titan Oracle doing?"})
print(response["answer"])
print('sources:')

sources = set()

for doc in response['context']:
    source = doc.metadata.get('link', 'Unknown')
    sources.add(source)

print(sources)


This article doesn't mention Oracle's performance.  It focuses on the negative performance of Tesla, U.S. Bancorp, and UnitedHealth Group, and the positive performance of Taiwan Semiconductor.  There is no information provided about Oracle.
sources:
{'https://apnews.com/article/stock-markets-inflation-data-earnings-42efcede3bc3db358117c72f2504edf2'}


In [20]:
response = rag_chain.invoke({"input": "is boing resuming production of its bestselling plane?"})
print(response["answer"])
print('sources:')

sources = set()

for doc in response['context']:
    source = doc.metadata.get('link', 'Unknown')
    sources.add(source)

print(sources)

The provided text doesn't offer any information about Boeing resuming production of its bestselling plane.  It focuses on broader market trends, Tesla's performance, and general economic indicators.  There's no mention of Boeing or its production plans.
sources:
{'https://apnews.com/article/stock-markets-inflation-data-earnings-42efcede3bc3db358117c72f2504edf2'}


## UI using gradio

In [21]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 24.1.0
    Uninstalling aiofiles-24.1.0:
      Successfully uninstalled aiofiles-24.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.29.0 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.


In [ ]:
import gradio as gr

def query_model(user_question):
    try:
        response = rag_chain.invoke({"input": user_question})
        answer = response.get("answer", "No answer available.")

        # Extract sources
        sources = set()
        for doc in response["context"]:
            source = doc.metadata.get("link", "Unknown")
            sources.add(source)

        sources_text = "\n".join(sources)
        return answer, sources_text
    except Exception as e:
        return f"An error occurred: {e}", "No sources available."

# Create the Gradio interface
interface = gr.Interface(
    fn=query_model,
    inputs=gr.Textbox(label="Ask a Question"),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Sources"),
    ],
    title="Equity Research Assistant",
    description="Ask questions about stock market and equity research. Powered by a retrieval-augmented generation (RAG) model."
)

# Launch the Gradio app
interface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://444548534270921aea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### References
- [Codebasics](https://www.youtube.com/watch?v=MoqgmWV1fm8&t=80s)
- [Document Loaders](https://www.comet.com/site/blog/langchain-document-loaders-for-web-data/)